# Assignment 3: Logistic Regression

In [2]:
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [3]:
#aug_D creation
def augmentedD(D):
    n = len(D)
    a0 = np.array([np.ones(n)]).T
    aug_D = np.hstack((a0,D))
    return aug_D

## Logistic Function

$$\theta(z) = \frac{1}{1 + e^{-z}}$$

In [4]:
def theta(z):
    den = 1 + np.exp(-z)
    return (1/den)

## Logistic Regression: Stochastic Gradient Ascent

![](logisticsga.png)


In [5]:
def logSGA(D, y, eta, eps):
    y = y[:, 0]
    D_aug = augmentedD(D)
    d = len(D_aug[0])
    n = len(D_aug)
    w_0 = np.array([np.zeros(d)]).T
    w_prev = w_0
    w_curr = w_0
    
    while True:
        
        w_prev = w_curr
        w = w_curr
            
            
        r = list(range(n))
        random.shuffle(r)  
        for i in r:
            x_i = np.array([D_aug[i,:]]).T
            grad = ( y[i] - theta( np.matmul(w.T, x_i)[0][0] ) )  * x_i
            w = w + eta * grad
            
        w_curr = w
        diff = w_curr - w_prev
        diff = diff[:, 0]

        if(np.dot(diff, diff)**.5 < eps):
            y_hat = np.matmul(D_aug, w_curr)
            return w, y_hat
            

## Classification

$\hat{y}_i = 1$ , if $\theta(w^Tz_j) \geq  .5$

$\hat{y}_i = 0$ , if $\theta(w^Tz_j) <  .5$


In [6]:
def binary_class(y_hat):
    y_hat = y_hat[:,0]
    n = len(y_hat)
    for i in range(n):
        if (theta(y_hat[i]) >= .5):
            y_hat[i] = 1
        else:
            y_hat[i] = 0
            
    return np.array([y_hat]).T

## Accuracy

$$acc = \frac{\text{number of cases where,  } \hat{y}_i = y_i}{n}$$

In [7]:
def accuracy(y, y_hat):
    y = y[:,0]
    y_hat = y_hat[:,0]
    count = 0
    for i in range(len(y)):
        if(y[i] == y_hat[i]):
            count = count + 1
    
    return count/len(y)

# Training Set

In [8]:
train = np.genfromtxt("Concrete_Data_RNorm_Class_train.txt", delimiter=',')


d = len(train[0])
D_train = train[:,np.arange(d-1)]
y_train = np.array([train[:,d-1]]).T

w , y_hat = logSGA(D_train, y_train, .013, 0.01)
    
y_hat = binary_class(y_hat)
acc = accuracy(y_train, y_hat)
print("Training Accuracy: ", acc)


Training Accuracy:  0.8143203883495146


# Testing Set

In [9]:
test = np.genfromtxt("Concrete_Data_RNorm_Class_test.txt", delimiter=',')
    
d = len(test[0])
D_test = test[:,np.arange(d-1)]
D_aug_test = augmentedD(D_test)

y_test = np.array([test[:,d-1]]).T

y_hat_test = np.matmul(D_aug_test, w)
y_hat_test = binary_class(y_hat_test)
acc = accuracy(y_test, y_hat_test)
print("Testing Accuracy:", acc)

Testing Accuracy: 0.8495145631067961


In [11]:
eta = .001
maximum =0
max_eta = 0
while (eta < .02):
    w , y_hat = logSGA(D_train, y_train, eta, 0.01)
    y_hat_test = np.matmul(D_aug_test, w)
    y_hat_test = binary_class(y_hat_test)
    acc = accuracy(y_test, y_hat_test)
    if(acc > maximum):
        maximum = acc
        max_eta = eta
    eta = eta + .001
print("Best Testing Accuracy: ", maximum)
print("Eta: ", max_eta)
print("With weights...")
print(w)

Best Testing Accuracy:  0.8543689320388349
Eta:  0.005
With weights...
[[-3.50406166]
 [ 6.21301368]
 [ 3.0710845 ]
 [ 1.68333333]
 [-4.2105264 ]
 [ 1.76824888]
 [ 0.1668656 ]
 [-0.86346841]
 [ 8.90162392]]


## Fixed value

Keeping $\epsilon$ fixed at .01 to keep each logistic regression run less than a second and testing values of $\eta$ from .001 to .02 we find that the best $\eta = 0.005$ with training accuracy of .85